In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password, username

# Extract

In [2]:
csv_path = "../Resources/customers.csv"
customer_df = pd.read_csv(csv_path)
customer_df.head()

,first_name,last_name,email,gender,car
0,Benetta,Cancott,bcancott0@studiopress.com,Female,Scion
1,Lilyan,Cherry,lcherry1@deliciousdays.com,Female,Chrysler
2,Ezekiel,Benasik,ebenasik2@wikia.com,Male,Mercedes-Benz
3,Kennedy,Atlay,katlay3@so-net.ne.jp,Male,Buick
4,Sanford,Salmen,ssalmen4@reuters.com,Male,Lincoln


In [3]:
json_path = "../Resources/addresses.json"
address_df = pd.read_json(json_path)
address_df.head()

,address,latitude,longitude,us_state,name
0,043 Mockingbird Place,39.1682,-86.5186,Indiana,"Cancott,Benetta"
1,4 Prentice Point,41.0938,-85.0707,Indiana,"Cherry,Lilyan"
2,46 Derek Junction,32.7673,-96.7776,Texas,"Benasik,Ezekiel"
3,11966 Old Shore Place,39.0350,-94.3567,Missouri,"Atlay,Kennedy"
4,5 Evergreen Circle,40.7808,-73.9772,New York,"Salmen,Sanford"


# Transform

## EDA
* You will need to do a whole lot more than I have done

In [4]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  1000 non-null   object
 1   last_name   1000 non-null   object
 2   email       1000 non-null   object
 3   gender      1000 non-null   object
 4   car         1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [5]:
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    1000 non-null   object 
 1   latitude   1000 non-null   float64
 2   longitude  1000 non-null   float64
 3   us_state   1000 non-null   object 
 4   name       1000 non-null   object 
dtypes: float64(2), object(3)
memory usage: 39.2+ KB


## Data Cleaning
* You will have to do a whole lot more than I have done

In [6]:
address_df[["last_name", "first_name"]] = address_df["name"].str.split(',', expand=True)

address_df.head()

,address,latitude,longitude,us_state,name,last_name,first_name
0,043 Mockingbird Place,39.1682,-86.5186,Indiana,"Cancott,Benetta",Cancott,Benetta
1,4 Prentice Point,41.0938,-85.0707,Indiana,"Cherry,Lilyan",Cherry,Lilyan
2,46 Derek Junction,32.7673,-96.7776,Texas,"Benasik,Ezekiel",Benasik,Ezekiel
3,11966 Old Shore Place,39.0350,-94.3567,Missouri,"Atlay,Kennedy",Atlay,Kennedy
4,5 Evergreen Circle,40.7808,-73.9772,New York,"Salmen,Sanford",Salmen,Sanford


In [7]:
merged_df = pd.merge(address_df, customer_df, on = ["last_name", "first_name"])
merged_df.head()

,address,latitude,longitude,us_state,name,last_name,first_name,email,gender,car
0,043 Mockingbird Place,39.1682,-86.5186,Indiana,"Cancott,Benetta",Cancott,Benetta,bcancott0@studiopress.com,Female,Scion
1,4 Prentice Point,41.0938,-85.0707,Indiana,"Cherry,Lilyan",Cherry,Lilyan,lcherry1@deliciousdays.com,Female,Chrysler
2,46 Derek Junction,32.7673,-96.7776,Texas,"Benasik,Ezekiel",Benasik,Ezekiel,ebenasik2@wikia.com,Male,Mercedes-Benz
3,11966 Old Shore Place,39.0350,-94.3567,Missouri,"Atlay,Kennedy",Atlay,Kennedy,katlay3@so-net.ne.jp,Male,Buick
4,5 Evergreen Circle,40.7808,-73.9772,New York,"Salmen,Sanford",Salmen,Sanford,ssalmen4@reuters.com,Male,Lincoln


## Get data into separate dataframes to match tables
* Make sure to include primary keys!

In [8]:
merged_df = merged_df.reset_index().rename(columns={"index": "customer_id"})
merged_df.head()

,customer_id,address,latitude,longitude,us_state,name,last_name,first_name,email,gender,car
0,0,043 Mockingbird Place,39.1682,-86.5186,Indiana,"Cancott,Benetta",Cancott,Benetta,bcancott0@studiopress.com,Female,Scion
1,1,4 Prentice Point,41.0938,-85.0707,Indiana,"Cherry,Lilyan",Cherry,Lilyan,lcherry1@deliciousdays.com,Female,Chrysler
2,2,46 Derek Junction,32.7673,-96.7776,Texas,"Benasik,Ezekiel",Benasik,Ezekiel,ebenasik2@wikia.com,Male,Mercedes-Benz
3,3,11966 Old Shore Place,39.0350,-94.3567,Missouri,"Atlay,Kennedy",Atlay,Kennedy,katlay3@so-net.ne.jp,Male,Buick
4,4,5 Evergreen Circle,40.7808,-73.9772,New York,"Salmen,Sanford",Salmen,Sanford,ssalmen4@reuters.com,Male,Lincoln


In [9]:
names_df = merged_df[["customer_id", "first_name", "last_name"]]
names_df.head()

,customer_id,first_name,last_name
0,0,Benetta,Cancott
1,1,Lilyan,Cherry
2,2,Ezekiel,Benasik
3,3,Kennedy,Atlay
4,4,Sanford,Salmen


In [10]:
location_df = merged_df[["customer_id", "address", "us_state"]]
location_df.head()

,customer_id,address,us_state
0,0,043 Mockingbird Place,Indiana
1,1,4 Prentice Point,Indiana
2,2,46 Derek Junction,Texas
3,3,11966 Old Shore Place,Missouri
4,4,5 Evergreen Circle,New York


In [11]:
location_df = location_df.reset_index().rename(columns={"index": "location_id"})
location_df.head()

,location_id,customer_id,address,us_state
0,0,0,043 Mockingbird Place,Indiana
1,1,1,4 Prentice Point,Indiana
2,2,2,46 Derek Junction,Texas
3,3,3,11966 Old Shore Place,Missouri
4,4,4,5 Evergreen Circle,New York


In [12]:
email_df = merged_df[["customer_id", "email"]]
email_df.head()

,customer_id,email
0,0,bcancott0@studiopress.com
1,1,lcherry1@deliciousdays.com
2,2,ebenasik2@wikia.com
3,3,katlay3@so-net.ne.jp
4,4,ssalmen4@reuters.com


In [13]:
email_df = email_df.reset_index().rename(columns={"index": "email_id"})
email_df.head()

,email_id,customer_id,email
0,0,0,bcancott0@studiopress.com
1,1,1,lcherry1@deliciousdays.com
2,2,2,ebenasik2@wikia.com
3,3,3,katlay3@so-net.ne.jp
4,4,4,ssalmen4@reuters.com


# Load

## Connect to local database

In [14]:
rds_connection_string = f"{username}:{password}@localhost:5432/customers_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for tables

In [15]:
engine.table_names()

['customer_email', 'customer_name', 'customer_location']

## Export data from dataframes to tables

In [16]:
names_df.to_sql("customer_name", con=engine, index=False, if_exists="append")

In [17]:
location_df.to_sql("customer_location", con=engine, index=False, if_exists="append")

In [18]:
email_df.to_sql("customer_email", con=engine, index=False, if_exists="append")

## Confirm data has been added by querying tables
* NOTE: can also check using pgAdmin

In [19]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,customer_id,first_name,last_name
0,0,Benetta,Cancott
1,1,Lilyan,Cherry
2,2,Ezekiel,Benasik
3,3,Kennedy,Atlay
4,4,Sanford,Salmen


In [20]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,location_id,customer_id,address,us_state
0,0,0,043 Mockingbird Place,Indiana
1,1,1,4 Prentice Point,Indiana
2,2,2,46 Derek Junction,Texas
3,3,3,11966 Old Shore Place,Missouri
4,4,4,5 Evergreen Circle,New York


In [21]:
pd.read_sql_query('select * from customer_email', con=engine).head()

,email_id,customer_id,email
0,0,0,bcancott0@studiopress.com
1,1,1,lcherry1@deliciousdays.com
2,2,2,ebenasik2@wikia.com
3,3,3,katlay3@so-net.ne.jp
4,4,4,ssalmen4@reuters.com
